In [67]:
from data_cleaning import merge_data
import pandas as pd

df = merge_data()
data = df[['Age', 'Country', 'self_employed', 'family_history',
               'no_employees', 'tech_company', 'wellness_program',
               'treatment']].dropna()

cat_cols = ['Country', 'self_employed', 'family_history', 'no_employees', 'tech_company', 'wellness_program', 'treatment']
df_encoded = pd.get_dummies(data, columns=cat_cols)
df_encoded

,Age,Country_Afghanistan,Country_Argentina,Country_Australia,Country_Austria,"Country_Bahamas, The",Country_Bangladesh,Country_Belgium,Country_Bosnia and Herzegovina,Country_Brazil,...,no_employees_6-25,no_employees_More than 1000,tech_company_No,tech_company_Yes,wellness_program_Don't know,wellness_program_I don't know,wellness_program_No,wellness_program_Yes,treatment_No,treatment_Yes
18,46,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
19,36,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,1,0
20,29,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
21,31,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
22,46,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,26,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
346,31,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
347,27,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,1,0
350,30,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0


In [68]:
import plotly.express as px

# Select only the variables with the treatment column
matrix_corr = df_encoded.corr()
matrix_corr

KeyError: 'treatment'